# Encontrando una política para jugar el 21

Esta libreta utiliza un algoritmo de programación dinámica para encontrar una política que permita jugar al 21.

Las reglas para este 21 son especiales:
- El mazo es infinito. De esta manera las 13 cartas siempre tienen las mismas probabilidades de aparecer
- El jugador realiza dos acciones antes de que el repartidor juegue
- Quien llegue a 4 cartas sin pasarse automáticamente gana

Con las reglas explicadas, definiremos una estructura para procesos de decisión markoviana.

In [ ]:
using StatsBase

In [ ]:
struct MDP
    states
    actions
    ρ
    reward
    final_s
end

## Funciones para hallar la política

Primero implementaremos una función que nos permitirá calcular el valor para cada estado de nuestro Proceso de Decisión de Markov con una política $\pi$ y con factor de descuento $\gamma$.

Esta función calcula iterativamente el valor para cada estado hasta que los resultados convergen.

In [ ]:
"""
    policy_value(mdp::MDP, pol_π::Dict, γ::Float64)

Computes the value for every state in the MDP by using pol_π and discount γ.
"""
function policy_value(mdp::MDP, pol_π::Dict, γ::Float64)
    v = Dict(s => 0.0 for s in mdp.states)
    
    has_converged = false
    while !has_converged
        has_converged = true
    
        for s in keys(v)
            temp = sum([mdp.ρ(s, pol_π[s], n_s) * (mdp.reward(s, pol_π[s], n_s) + γ*v[n_s]) for n_s in keys(v)])
                    
            if temp != v[s]
                has_converged = false
            end
            
            v[s] = temp
        end
    end
            
    return v
end

Con la función anterior podemos programar ahora una función que encuentre una política óptima para un Proceso de Decisión de Markov.

In [ ]:
"""
    policy_iteration(mdp::MDP, γ)

Finds an optimal policy for the MDP using discount factor γ.
"""
function policy_iteration(mdp::MDP, γ)
    pol_π = Dict(s => sample(mdp.actions) for s in mdp.states)
    
    is_optimal = false
    while !is_optimal
        v = policy_value(mdp, pol_π, γ)
        
        is_optimal = true
        
        for s in keys(v)
            for a in mdp.actions
                temp = sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ*v[s]) for n_s in keys(v)])
                
                if temp < v[s]
                    is_optimal = false
                    pol_π[s] = a
                end
            end
        end
    end
    
    return pol_π
end

Enseguida viene una implementación de un algoritmo iterativo para encontrar una buena política, todo en un único bloque de código.

In [ ]:
"""
    iter_value(mdp::MDP, γ::Float64)

Iteratively computes the value function of a Markov Decision Process using
discount rate γ and then returns the optimal policy π associated with it.
"""
function iter_value(mdp::MDP, γ::Float64)
    v = Dict(s => 0.0 for s in mdp.states)
    v_p = Dict(s => 0.0 for s in mdp.states)
    
    has_converged = false
    
    while !has_converged
        for s in keys(v)
            v_p[s] = maximum([sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ * v[n_s])
                                        for n_s in mdp.states])
                                    for a in mdp.actions])

            has_converged = true
                            
            for s in keys(v)
                if v_p[s] > v[s]
                    v[s] = v_p[s]
                    has_converged = false
                end
            end
            
            if has_converged
                break
            end
        end
    end
    
    pol_π = Dict(s => "" for s in mdp.states)
    
    for s in keys(v)
        actions_value = Dict(a => sum([mdp.ρ(s, a, n_s) * v[n_s] for n_s in mdp.states])
                            for a in mdp.actions)
                                
        pol_π[s] = findmax(actions_value)[2]
    end
    
    return pol_π
end

## Definiendo el proceso de decisión de Markov

En las siguientes celdas viene el código necesario para definir un proceso de decisión de Markov que represente el juego y que sea compatible con ``policy_value``, ``policy_iteration``, ``iter_value``.

Primero definiremos los estados del juego como un arreglo de dos de números enteros.

$$(S, C)$$

Donde $S$ es la suma total de la cantidad $C$ de cartas que tiene el jugador.

In [ ]:
states = [[2, i] for i in 2:26]

for i in 3:39
    push!(states, [3, i])
end

for i in 4:52
    push!(states, [4, i])
end

Ahora las acciones. Esta parte es fácil, solo hay dos posibles acciones en todo momento. Esta libreta usa el vocabulario usado en los casinos.

In [ ]:
actions = ["hit", "stand"]

Seguimos con la declaración de la función que calcula la probabilidad de transición entre estados.

In [ ]:
function ρ(s, a, n_s)
    if a == "stand"
        if s == n_s
            return 1
        else
            return 0
        end
    else
        diff_score = n_s[2] - s[2]
        
        if n_s[1] == s[1] + 1 && diff_score >= 1 && diff_score <= 13 
            return 1/13
        end
    end
    
    return 0
end

Casi terminando, calculamos la recompensa para cada estado. Aquí no hay ganancia ni pérdida a menos que el juego se acabe.

In [ ]:
function reward(s, a, n_s)
    if (n_s[1] == 4 && n_s[2] <= 21) || n_s[2] == 21
        return 1
    elseif n_s[2] > 21
        return -1
    else
        return 0
    end
end

## Resolviendo el problema

Como tenemos todos los preparativos listos, podemos crear un nuevo proceso de decisión de Markov que modele este 21.

In [ ]:
twenty_one = MDP(states, actions, ρ, reward, [])

Y con esto, ahora podemos probar si nuestro algoritmo realmente cumple su tarea.

In [ ]:
γ = 0.8
pol_π = policy_iteration(twenty_one, γ)

In [ ]:
γ = 0.8
pol_π = iter_value(twenty_one, γ)

In [ ]:
policy_value(twenty_one, pol_π, γ)

Veamos los resultados. No muy buenos por ahora.

In [ ]:
for s in states
    println(s, pol_π[s])
end